<a href="https://colab.research.google.com/github/gurunars/ipythons/blob/main/homm_creatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import json
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import defaultdict

ROOT = "https://heroes.thelazy.net"

PATH = ROOT + "/index.php/List_of_creatures_(HotA)"

def get_creatures():
  html_doc = requests.get(PATH).text

  soup = BeautifulSoup(html_doc, 'html.parser')

  rows = soup.select('tr')

  for row in rows:
    cols = row.select('td')
    if not cols:
      continue

    yield dict(
      icon_url = ROOT + cols[0].select('img')[0].attrs['src'].strip(),
      castle_url = ROOT + cols[1].select('img')[0].attrs['src'].strip(),
      castle_name = cols[1].select('span')[0].attrs['title'].strip(),
      level = cols[2].select('span')[0].text.strip(),
      name = cols[0].select('a')[-1].text.strip(),
      hp = int(cols[7].select('span')[0].text.strip()),
      cost = int(cols[11].text.strip())
    )

def get_grouped_creatures():
  grouped = defaultdict(list)
  for creature in sorted(get_creatures(), key=lambda it: it["level"]):
    grouped[(creature["castle_name"], creature["castle_url"])].append(creature)
  return grouped



In [4]:
from google.colab import files

with open('creatures.json', 'w') as fil:
  json.dump(list(get_creatures()), fil, indent=2)

files.download('creatures.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
from jinja2 import Environment, BaseLoader

TEMPLATE = """
<table>
  <tr>
    <th align="left" style="text-align: left;">Creature</th>
    <th align="left" style="text-align: left;">Level</th>
    <th align="left" style="text-align: left;">HP</th>
    <th align="left" style="text-align: left;">Cost</th>
  </tr>

  {% for faction, creatures in grouped_creatures.items()%}
    <tr style="background-color: Gainsboro;;">
      <td colspan="4">
        <span style="text-align: center; display: flex; justify-content: center; align-items: center;">
          <img src="{{faction[1]}}"/>
          &nbsp;
          <b>{{faction[0]}}</b>
        </span>
      </td>
    </tr>
    {% for creature in creatures %}
      <tr>
        <td align="left" style="text-align: left;"><img src="{{creature.icon_url}}"/> {{creature.name}}</td>
        <td align="left" style="text-align: left;">{{creature.level}}</td>
        <td align="left" style="text-align: left;">{{creature.hp}}</td>
        <td align="left" style="text-align: left;">{{creature.cost}}</td>
      </tr>
    {% endfor %}
  {% endfor %}
</table>
"""

TPL = Environment(loader=BaseLoader()).from_string(TEMPLATE)

def get_creatures_as_html():
  return TPL.render(grouped_creatures=get_grouped_creatures())

HTML(get_creatures_as_html())